# Using a Neural Network approach to see whether an author's name constitutes a critical/commercial hit or flop:


In [3]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import functions
from sklearn.preprocessing import LabelEncoder
%pip install torch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

Note: you may need to restart the kernel to use updated packages.


Checking the amount of unique authors in the dataset. Credit to ChatGPT for showing how to do so:


In [2]:
df_metadata = functions.get_data()
df_metadata['author_name'].nunique()

12877

Over 12,000 authors. Statiscally, most authors have 1 book, so I will check if that is the case. Credit to ChatGPT which showed me how to code up a solution to this:

In [3]:
df_metadata['author_name'].value_counts().describe()

count    12877.000000
mean         1.553157
std          1.800488
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         76.000000
Name: count, dtype: float64

This shows that the majority of authors have only 1 book to their names making it likely that they are debut authors. Lastly, it can be shown that the maximum number of books an author has got on the Amazon storefront is 76. This shows that author name alone is not enough to determine a critical or commercial success. As such, my neural network will use the context of this information to determine whether the general audience would support a given book both financially and critically. 

## The Foundations of the Neural Network:

Credit to ChatGPT for showing me how to lay the groundwork for my neural network:

In [4]:
# Start by calling my testing and training data:
X_train, X_test, y_train, y_test = functions.test_train_split()

# Need to convert to Pytorch tensors:
X_train = torch.tensor(X_train, dtype = torch.long)
X_test = torch.tensor(X_test, dtype = torch.long)
y_train = torch.tensor(y_train, dtype = torch.float32)
y_test = torch.tensor(y_test, dtype = torch.float32)

# Making the Dataset and Dataloader:
train_ds = torch.utils.data.TensorDataset(X_train, y_train)
test_ds = torch.utils.data.TensorDataset(X_test, y_test)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size = 32, shuffle = True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size = 32)

# Defining the Model:
class AuthorNN(nn.Module):
    def __init__(self, num_authors, embed_dim = 8):
        super().__init__()
        self.embedding = nn.Embedding(num_authors, embed_dim)
        self.fc1 = nn.Linear(embed_dim, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x = x.view(x.size(0), -1) # Flattens the model
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

num_authors = len(set(X_train.numpy()))

model = AuthorNN(num_authors)
criterion = nn.BCELoss()
optimiser = optim.Adam(model.parameters(), lr = 0.001)

# Making the training loop:
for epoch in range(10):
    model.train()
    epoch_loss = 0
    for xb, yb in train_dl:
        optimiser.zero_grad()
        preds = model(xb).squeeze()
        loss = criterion(preds, yb)
        loss.backward()
        optimiser.step()
        epoch_loss += loss.item()
    print(f"Epoch, {epoch+1}: loss = {epoch_loss:.3f}")

# Evaluation:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for xb, yb in test_dl:
        preds = (model(xb).squeeze() > 0.5).float()
        correct += (preds == yb).sum().item()
        total += yb.size(0)

print("Test accuracy:", correct / total)

IndexError: index out of range in self

Index(['author_name', 'publisher', 'publisher_date', 'format', 'page_count',
       'language', 'category_level_2_sub', 'category_level_3_detail',
       'average_rating', 'rating_number', 'price_numeric', 'maybe_date',
       'author_id'],
      dtype='object')